# Проект по SQL

#### Компания купила крупный сервис для чтения книг по подписке. Первая задача — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Задания

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — для исключения из анализа брошюр;
- Определить автора с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками;
- Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                     db_config['pwd'],
                     db_config['host'],
                     db_config['port'],
                     db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

#### 1 Посчитаем, сколько книг вышло после 1 января 2000 года.

In [3]:
query = '''SELECT COUNT(DISTINCT book_id) AS "количество"
           FROM books
           WHERE publication_date > '2000-01-01'
           '''

In [4]:
result = pd.io.sql.read_sql(query, con = engine) 

In [5]:
print(result)

   количество
0         819


После 1 января 2000 года вышло 819 книг.

#### 2 Посчитаем количество обзоров и среднюю оценку для каждой книги.

In [6]:
query = '''SELECT title AS "название", COUNT(DISTINCT review_id) AS "количество рецензий",
           ROUND(AVG(rating), 2) AS "средняя оценка"
           FROM reviews INNER JOIN ratings ON reviews.book_id=ratings.book_id
                        RIGHT JOIN books ON ratings.book_id=books.book_id
           GROUP BY books.title
           '''

In [7]:
result = pd.io.sql.read_sql(query, con = engine) 

In [8]:
print(result)

                                              название  количество рецензий  \
0                                         'Salem's Lot                    2   
1                   1 000 Places to See Before You Die                    1   
2    13 Little Blue Envelopes (Little Blue Envelope...                    3   
3    1491: New Revelations of the Americas Before C...                    2   
4                                                 1776                    4   
..                                                 ...                  ...   
994           Wyrd Sisters (Discworld  #6; Witches #2)                    3   
995                        Xenocide (Ender's Saga  #3)                    3   
996                                    Year of Wonders                    4   
997                        You Suck (A Love Story  #2)                    2   
998  Zen and the Art of Motorcycle Maintenance: An ...                    4   

     средняя оценка  
0              3.67  
1      

Для каждой книги найдено количество обзоров и средняя оценка.

#### 3 Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [9]:
query = '''SELECT publisher AS "издательство", COUNT(book_id) AS "количество книг толще 50-ти страниц"
           FROM publishers INNER JOIN (
                                      SELECT *
                                      FROM books
                                      WHERE num_pages > 50
                                      ) AS big_books
                                      ON publishers.publisher_id=big_books.publisher_id
           GROUP BY publisher
           ORDER BY 2 DESC
           LIMIT 1
                      
           '''

In [10]:
result = pd.io.sql.read_sql(query, con = engine)

In [11]:
print(result)

    издательство  количество книг толще 50-ти страниц
0  Penguin Books                                   42


Наибольшее число книг толще 50 страниц выпустило издательство Penguin Books (42 шт).

#### 4 Определим автора с самой высокой средней оценкой книг, учитываем книги с 50-ю и более оценками.

In [12]:
query = '''SELECT author AS "автор", round(AVG(avg), 2) AS "средняя оценка книг"
           FROM authors INNER JOIN books ON authors.author_id=books.author_id
                        INNER JOIN (
                                   SELECT book_id, AVG(rating)
                                   FROM ratings
                                   GROUP BY book_id
                                   HAVING COUNT(rating_id) > 50
                                   ) AS rated_books
                                   ON books.book_id=rated_books.book_id
           GROUP BY author
           ORDER BY 2 DESC
           LIMIT 1
                      
           '''

In [13]:
result = pd.io.sql.read_sql(query, con = engine)

In [14]:
print(result)

                        автор  средняя оценка книг
0  J.K. Rowling/Mary GrandPré                 4.28


 Автор с самой высокой средней оценкой книг, учитывая книги с 50-ю и более оценками - J.K. Rowling/Mary GrandPré (4.28).

#### 5 Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50-ти оценок.

In [15]:
query = '''SELECT ROUND(AVG(count)) AS "среднее количество обзоров"
           FROM (
                SELECT COUNT(review_id)
                FROM reviews INNER JOIN (
                                        SELECT username
                                        FROM ratings
                                        GROUP BY username
                                        HAVING COUNT(rating_id) > 50
                                        ) AS active_users
                                        ON reviews.username=active_users.username                                   
                GROUP BY active_users.username
                ) AS active_users_cnt_rev         
                                
               '''

In [16]:
result = pd.io.sql.read_sql(query, con = engine)

In [17]:
print(result)

   среднее количество обзоров
0                        24.0


Среднее количество обзоров от пользователей, поставивших больше 50-ти оценок - 24 шт.

### Итоговый вывод:

- После 1 января 2000 года вышло 819 книг.
- Для каждой книги найдено количество обзоров и средняя оценка.
- Наибольшее число книг толще 50 страниц выпустило издательство Penguin Books (42 шт).
- Автор с самой высокой средней оценкой книг, учитывая книги с 50-ю и более оценками - J.K. Rowling/Mary GrandPré (4.28).
- Среднее количество обзоров от пользователей, поставивших больше 50-ти оценок - 24 шт.